In [2]:
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb
%run make_features.ipynb
from pathlib import Path, PurePath
import numpy as np
import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch


pick_ch=['P9','P7','P5','P3','PO7','PO3','O1','P10','P8','P6','P4','PO8','PO4','O2','POz','Pz']
#pick_ch=['PO8','PO7']

infolder, outfolder = find_folder()
trigs=list(range(101, 151))+list(range(201, 251))
event_ids={str(x):x for x in trigs}
im_times=(-0.1,1)
filt=(0.1,110)
fnames = ['E:\\Ilya_study\\Data\\IR_05_S01.bdf','E:\\Ilya_study\\Data\\IR_05_S02.bdf']
epochs = load_to_epochs_perc(fnames, event_ids, im_times, filt)
epochs.drop_channels(['Status']).equalize_event_counts(event_ids=event_ids, method='mintime')
epochs = block_average(epochs,4,12, kind='perc',zscore = False)
pick_ch = mne.pick_channels(epochs.ch_names, pick_ch)

Extracting EDF parameters from E:\Ilya_study\Data\IR_05_S01.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3388415  =      0.000 ...  6617.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1.1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 110.00 Hz: -6.02, -6.02 dB

2500 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 2500 events and 564 original time points ...
0 bad epochs dropped
Extracting EDF parameters from E:\Ilya_study\Data\IR_05_S02.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3387903  =      0.000 ...  6616.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pa

In [5]:
psds, freqs = psd_multitaper(epochs, fmin=1, fmax=3, n_jobs=1, picks = pick_ch)
psds, freqs = psd_multitaper(epochs, fmin=1, fmax=10, n_jobs=1)
ffts = np.reshape(psds,(psds.shape[0],-1),order='F')
print(psds.shape)

from statsmodels.tsa.ar_model import AutoReg
lags = 10
data = epochs.get_data()[:, pick_ch, :]
temp = np.full((data.shape[0],data.shape[1],lags+1), np.nan)

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        model = AutoReg(data[i,j,:], lags=lags)
        model_fit = model.fit()
        temp[i,j,:] = np.array(model_fit.params)
        
Y = epochs.events[:, 2]
X = np.reshape(temp,(temp.shape[0],temp.shape[1]*temp.shape[2]),order='F')
X = np.hstack((X,ffts))
X = ffts
confusion, duration = run_eeg_svm(X, Y, 12)

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
(1200, 64, 10)
No PCA
No fft
The overall upright accuracy is 50.7 and the duration is 1.9 minutes


In [11]:
X = np.hstack((np.reshape(data,(1200,16*564),order='F'),ffts))
X = np.reshape(data,(1200,16*564),order='F')
confusion, duration = run_eeg_svm(X, Y, 12)

No PCA
No fft
The overall upright accuracy is 54.5 and the duration is 11.7 minutes


In [25]:
epochs.get_data().shape

(1200, 64, 282)

In [41]:
from pathlib import Path, PurePath
prc_time = (0.05,0.65)
infolder, outfolder = find_folder()
pick_ch=['P9','P7','P5','P3','PO7','PO3','O1','P10','P8','P6','P4','PO8','PO4','O2','POz','Pz']
fname = PurePath(outfolder,'S'+str(5).zfill(2)+'_perc_aver-epo.fif')
epochs = mne.read_epochs(fname)
psds, freqs = psd_multitaper(epochs, fmin=1, fmax=12, n_jobs=1, picks = mne.pick_channels(epochs.ch_names, pick_ch))
ffts = np.reshape(psds,(psds.shape[0],-1),order='F')
X, Y = convert_epochs_to_2D_array(epochs, times = prc_time, pick_ch=pick_ch)
X = np.hstack((X[:,::2],ffts))
confusion, duration = run_eeg_svm(X, Y, 12)

Reading E:\Ilya_study\Analysis\S05_perc_aver-epo.fif ...
    Found the data of interest:
        t =    -101.56 ...     996.09 ms
        0 CTF compensation matrices available
1200 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows


C:\Users\danne\Anaconda3\envs\eeg_analysis\lib\site-packages\sklearn\preprocessing\data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


No PCA
No fft
The overall upright accuracy is 55.1 and the duration is 1.7 minutes


In [53]:
from scipy.spatial.distance import squareform
a = confusion[:100,:100]
np.mean(squareform(a))

0.5639057239057238

In [13]:
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb
%run make_features.ipynb
from pathlib import Path, PurePath
import numpy as np
import mne
pick_chs=['P9','P7','P5','P3','PO7','PO3','O1','P10','P8','P6','P4','PO8','PO4','O2','POz','Pz']
#pick_ch=[]
infolder, outfolder = find_folder()
trigs=list(range(101, 151))+list(range(201, 251))
event_ids={str(x):x for x in trigs}
im_times=(-0.1,1)
filt=(0.1,110)
fnames = ['E:\\Ilya_study\\Data\\IR_05_S01.bdf','E:\\Ilya_study\\Data\\IR_05_S02.bdf']
iter_freqs = [
    (4, 7),
    (8, 12),
    (13, 25),
    (26, 45)
]
from statsmodels.tsa.ar_model import AutoReg
lags = 10

out = list()
for freq in iter_freqs:
    epochs = load_to_epochs_perc(fnames, event_ids, im_times, freq)
    epochs.drop_channels(['Status']).equalize_event_counts(event_ids=event_ids, method='mintime')
    epochs = block_average(epochs,4,12, kind='perc',zscore = False)
    pick_ch = mne.pick_channels(epochs.ch_names, pick_chs)
    data = epochs.get_data()[:, pick_ch, :]
    print(data.shape)
    temp = np.full((data.shape[0],data.shape[1],lags+1), np.nan)

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            model = AutoReg(data[i,j,:], lags=lags)
            model_fit = model.fit()
            temp[i,j,:] = np.array(model_fit.params)

    Y = epochs.events[:, 2]
    X = np.reshape(temp,(temp.shape[0],temp.shape[1]*temp.shape[2]),order='F')
    out.append(X)

Extracting EDF parameters from E:\Ilya_study\Data\IR_05_S01.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3388415  =      0.000 ...  6617.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 7 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 7.00 Hz: -6.02, -6.02 dB

2500 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 2500 events and 564 original time points ...
0 bad epochs dropped
Extracting EDF parameters from E:\Ilya_study\Data\IR_05_S02.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3387903  =      0.000 ...  6616.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter 

In [25]:
X = np.transpose(np.array(out), axes=(1,0,2))
X = np.reshape(X,(X.shape[0],-1),order='F')
confusion, duration = run_eeg_svm(X, Y, 12)

No PCA
No fft
The overall accuracy is 51.3 and the duration is 1.3 minutes


In [19]:
pick_ch

array([20, 21, 22, 23, 24, 25, 26, 29, 30, 57, 58, 59, 60, 61, 62, 63])

In [ ]:

X, Y = convert_epochs_to_2D_array(epochs, times = im_times, pick_ch=pick_ch)

In [ ]:
epochs

In [ ]:
def block_average(epochs, num_trials, num_blocks, kind, zscore = False, threshold = False):
    ''' Z-scores within time X els, replaces threshshold (if provided) and averages 
        trials within blocks. 
    :param epochs: epochs structure
    :param num_trials: number of trials to average
    :param num_blocks: number of blochs in the data
    :param items_n: number of unique triggers
    :param zscore: to perform zscoring across time X els. Default - False
    :param threshold: the threshold absolute z-value. Everything above is threshold, 
                        under is -threshold. Default - False
    :return epochs: - MNE epochs structure 
    
    >>> epochs_02 = block_average(epochs_02,4,11, zscore = True, threshold = 3)
    '''
    import pandas as pd
    from scipy import stats
    import mne
    import numpy as np
    
    if kind == 'perc':
        items_n = 100
    else:
        items_n = 5
    print('Starting averaging')
    df = epochs.to_data_frame()
    df = df.unstack(level = -1)
    df['block']=np.tile(np.arange(1,num_blocks+1).repeat(num_trials),items_n)
    df.reset_index(inplace=True)
    df['condition']=df['condition'].apply(pd.to_numeric)
    if zscore:
        arr = df.iloc[:,2:-1].values
        arr = stats.zscore(arr,axis=1)
        if threshold:
            arr[np.where(arr>threshold)]=threshold
            arr[np.where(arr<-threshold)]=-threshold
        else:
            print('No thresholding performed')
        df.iloc[:,2:-1]=arr
    else:
        print('No zscoring performed')
    df=df.groupby(['block','condition']).mean()
    df.reset_index(inplace=True)
    data = np.array(df.iloc[:,3:].values)
    data = data.reshape(data.shape[0],64,int(data.shape[1]/64))
    
    if kind == 'perc':
        east=list(np.arange(101,126))+list(np.arange(201,226))
        df['orientation']=np.where(df['condition']>200,'inv','up')
        df['origin']=np.where(df['condition'].isin(east),'east','west')
        trigs=list(range(101, 151))+list(range(201, 251))
    else: 
        trigs=list(range(31, 36))
    event_ids={str(x):x for x in trigs}
        
    # Initialize an info structure
    events = np.array([np.arange(len(df.condition)),np.zeros(len(df.condition),),df.condition]).transpose()
    events = events.astype('int')
    epochs = mne.EpochsArray(data, info=epochs.info, events=events, tmin = epochs.tmin)
    epochs.apply_baseline()
    '''
    if kind == 'perc':
        epochs.metadata = df[['block','condition','orientation','origin']]
    else:
        epochs.metadata = df[['block','condition']]
        '''
    return epochs

In [ ]:
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb
%run make_features.ipynb
from pathlib import Path, PurePath
import numpy as np
import mne
pick_ch=['P9','P7','P5','P3','PO7','PO3','O1','P10','P8','P6','P4','PO8','PO4','O2','POz','Pz']
pick_ch=['PO8']
infolder, outfolder = find_folder()
trigs=list(range(101, 151))+list(range(201, 251))
event_ids={str(x):x for x in trigs}
im_times=(-0.1,1)
filt=(0.1,40)
fnames = ['D:\\Ilya_study\\Data\\IR_05_S01.bdf','D:\\Ilya_study\\Data\\IR_05_S02.bdf']
epochs = load_to_epochs_perc(fnames, event_ids, im_times, filt)
epochs.drop_channels(['Status']).equalize_event_counts(event_ids=event_ids, method='mintime') 
#epochs = block_average(epochs,4,12, kind='perc',zscore = False)
X, Y = convert_epochs_to_2D_array(epochs, times = im_times, pick_ch=pick_ch)
ffts = np.abs(np.fft.fft(X))[:,:40]
print(ffts.shape)
data = epochs.get_data()
from statsmodels.tsa.ar_model import AutoReg
lags = 20
temp = np.full((data.shape[0],data.shape[1],lags+1), np.nan)

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        model = AutoReg(data[i,j,:], lags=lags)
        model_fit = model.fit()
        temp[i,j,:] = np.array(model_fit.params)
        
Y = epochs.events[:, 2]
X = np.reshape(temp,(temp.shape[0],temp.shape[1]*temp.shape[2]),order='F')
X = np.hstack((X,ffts))
confusion, duration = run_eeg_svm(X, Y, 12)

In [ ]:
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb
%run make_features.ipynb
from pathlib import Path, PurePath
import numpy as np
import mne
infolder, outfolder = find_folder()
trigs=list(range(101, 151))+list(range(201, 251))
event_ids={str(x):x for x in trigs}
im_times=(-0.1,1)
filt=(8,20)
fnames = ['D:\\Ilya_study\\Data\\IR_07_S01.bdf','D:\\Ilya_study\\Data\\IR_07_S02.bdf']
epochs = load_to_epochs_perc(fnames, event_ids, im_times, filt)
epochs.drop_channels(['Status']).equalize_event_counts(event_ids=event_ids, method='mintime') 
#epochs = block_average(epochs,4,12, kind='perc',zscore = False)
epochs

In [ ]:
epochs['101'].plot()

In [ ]:
a = data[0,0,:]
model = AutoReg(a, lags=lags)
model_fit = model.fit()
print('Coefficients: %s' % model_fit.params)

In [ ]:
data = zscore_threshold(data, dims, threshold)
epochs = mne.EpochsArray(data, epochs.info, events=epochs.events, tmin = epochs.tmin)

In [ ]:
%run general_tools.ipynb
%run EEG_auxiliary_module.ipynb
%run make_features.ipynb
from pathlib import Path, PurePath
import numpy as np
import mne
infolder, outfolder = find_folder()
pick_ch=['P9','P7','P5','P3','PO7','PO3','O1','P10','P8','P6','P4','PO8','PO4','O2','POz','Pz']

subs = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
subs = [3]
prc_time = (0.05,0.65)
data=list()
labels = list()
results=[]# Loading first S02 session
durs=[]
dictr = {PurePath(outfolder,"ar_prc_accuracy.pkl"):results, 
         PurePath(outfolder,"ar_prc_duration.pkl"):durs}
for i,j in enumerate(subs):
    update_progress(i/len(subs))
    fname = PurePath(outfolder,'S'+str(j).zfill(2)+'_perc_aver-epo.fif')
    epochs = mne.read_epochs(fname)
    cv=len(epochs['101'])
    X, Y = convert_epochs_to_2D_array(epochs, times = prc_time, pick_ch=pick_ch)
    #X,Y = epochs_to_stft_features(outfolder, fname, step = 76, times = prc_time, pick_ch = pick_ch)
    #X,Y = epochs_to_morlet_features(fname,times = prc_time,pick_ch = pick_ch)
    confusion, duration = run_eeg_svm(X, Y, cv)
    results.append(confusion)
    durs.append(duration)
    save_all_pickle(dictr = dictr)


In [ ]:
%reset -f

import sys
import mne
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pathlib import Path, PurePath

pick_ch=['P9','P7','P5','P3','PO7','PO3','O1','P10','P8','P6','P4','PO8','PO4','O2','POz','Pz']

%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb
mne.__version__
mne.set_log_level("WARNING")

infolder, outfolder = find_folder()
#outfolder = 'D:\\Ilya_study\\Analysis'
subs = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
subs = ['03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18']